In [1]:
import sys
import os
import warnings
import numpy as np
import tensorflow as tf
import SimpleITK as sitk
from nilearn import image, plotting

sys.path.append("../")
sys.path.append("../deepneuroan")
warnings.filterwarnings('ignore')

from deepneuroan.data_generator import DataGenerator
from deepneuroan.models import ChannelwiseConv3D, rigid_concatenated
from deepneuroan.generate_train_data import transform_volume
from deepneuroan.preproc import create_ref_grid

/home/ltetrel/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
E0120 16:43:03.162842 140562484008768 due.py:63] Failed to import duecredit due to No module named 'duecredit'


In [2]:
def dice_loss(y_true, y_pred):
  
    numerator = 2 * np.sum(y_true * y_pred)
    denominator = np.sum(y_true) + np.sum(y_pred)
    print(numerator)
    print(denominator)
    
    return (numerator + 1) / (denominator + 1)

In [3]:
data_dir = "/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/"
template_filepath = os.path.join(data_dir, "template_on_grid")

list_files = []
list_files_tmp = set([])
for root, _, files in os.walk(data_dir):
    for file in files:
        filepath = os.path.join(root, file).split('.')[0]
        if os.path.exists(filepath + ".txt"):
            list_files_tmp.add(filepath)
list_files = list(list_files_tmp)

bs = 2
ncpu = 1
np.random.seed(0)
params_gen = dict(list_files=list_files, template_file=template_filepath, batch_size=bs, avail_cores=ncpu)
valid_gen = DataGenerator(partition="valid", **params_gen)

In [4]:
with open("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/rigid_concatenated_2020-01-17_22:26:33.json", "r") as json_file:
    model = tf.keras.models.model_from_json(json_file.read(), custom_objects={'ChannelwiseConv3D': ChannelwiseConv3D})
model.load_weights("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/checkpoints/rigid_concatenated_best/rigid_concatenated_cp-0250.ckpt")
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01)
                  , loss=tf.keras.losses.mean_squared_error
                  , metrics=["mae"])

In [5]:
ref_grid = create_ref_grid()

# tmp
batch = 0

print(valid_gen.get_files_batch(0))
fixed = valid_gen.__getitem__(0)[0][:, :, :, :,0]
moving = valid_gen.__getitem__(0)[0][:, :, :, :,1]
pred = sitk.GetImageFromArray(moving[batch,])
pred.SetOrigin(ref_grid.GetOrigin())
pred.SetSpacing(ref_grid.GetSpacing())
pred.SetDirection(ref_grid.GetDirection())
# t_rigid = model.predict(x=valid_gen.__getitem__(0)[0], use_multiprocessing=False, verbose=0)[batch,]
t_rigid = valid_gen.__getitem__(0)[1][batch,]
registered_pred = transform_volume(pred, ref_grid, interp=sitk.sitkLinear, rigid=t_rigid)
registered_pred = sitk.GetArrayFromImage(registered_pred)
pred_mask = registered_pred > 0
truth_mask = fixed[batch,] > 0
# for i in range(all_gen.__len__()):
#     fixed = valid_gen.__getitem__(0)[0][:, :, :, :,0]
#     moving = valid_gen.__getitem__(0)[0][:, :, :, :,1]
#     for batch in range(valid_gen.batch_size):
#         sitk.GetImageFromArray(valid_gen.__getitem__(0)[0][:, :, :, :,0])
#         pred = sitk.Image(ref_grid.GetSize(), ref_grid.GetPixelIDValue())
#         pred.SetOrigin(ref_grid.GetOrigin())
#         pred.SetSpacing(ref_grid.GetSpacing())
#         pred.SetDirection(ref_grid.GetDirection())

['/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-0201_transfo-000003', '/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-0287_transfo-000008']


In [6]:
dice_loss(truth_mask, pred_mask)

1836402
3866094


0.47500203693908194

In [7]:
display = plotting.plot_anat("/home/ltetrel/Documents/work/DeepNeuroAN/data/MNI152_T1_1mm_brain")
display.add_edges("/home/ltetrel/Documents/data/%s_to_grid.nii.gz" %t_name[:6])

In [9]:
help(plotting.plot_anat)

Help on function plot_anat in module nilearn.plotting.img_plotting:

plot_anat(anat_img=<MNI152Template>, cut_coords=None, output_file=None, display_mode='ortho', figure=None, axes=None, title=None, annotate=True, threshold=None, draw_cross=True, black_bg='auto', dim='auto', cmap=<matplotlib.colors.LinearSegmentedColormap object at 0x7fd6cb42fa20>, vmin=None, vmax=None, **kwargs)
    Plot cuts of an anatomical image (by default 3 cuts:
    Frontal, Axial, and Lateral)
    
    Parameters
    ----------
    anat_img : Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        The anatomical image to be used as a background. If None is
        given, nilearn tries to find a T1 template.
    cut_coords : None, a tuple of floats, or an integer
        The MNI coordinates of the point where the cut is performed
        If display_mode is 'ortho' or 'tiled',
        this should be a 3-tuple: (x, y, z)
        For display_mode == 'x', 'y', or 'z', the